In [2]:
from importlib.metadata import version

print(version('torch'))

2.5.1


In [3]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

In [4]:
# attention score
query = inputs [1]
attn_scores_2 = torch.empty(inputs.shape[0]) # same as rows in inputs
for i , x_i in enumerate(inputs) :
    attn_scores_2[i] = torch.dot(x_i, query)

print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [6]:
atten_weights_2 = attn_scores_2 / attn_scores_2.sum()
print(atten_weights_2)
print(atten_weights_2.sum())

tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
tensor(1.0000)


In [7]:
# softmax from scratch
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

attn_wrights_2_naive = softmax_naive(attn_scores_2)
print(attn_wrights_2_naive)
print(attn_wrights_2_naive.sum())

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
tensor(1.)


In [8]:
# pytorch softmax

attn_weights_2 = torch.softmax(attn_scores_2, dim = 0)
print(attn_weights_2, attn_weights_2.sum())

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581]) tensor(1.)


In [10]:
# context vector calcualtion for query 2
query = inputs[1]
context_vec_2 = torch.zeros(query.shape)
for i, x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i] * x_i
print(context_vec_2)


tensor([0.4419, 0.6515, 0.5683])


In [12]:
# context vector for all elements
attn_scores = torch.empty(6,6)
for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
         attn_scores[i,j] = torch.dot(x_i , x_j)

print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [14]:
attn_weigths = torch.softmax(attn_scores, dim = 1)
print(attn_weigths)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [15]:
# alternative 
attn_scores = inputs @ inputs.T
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [19]:
attn_weigths = torch.softmax(attn_scores, dim =-1)
print(attn_weigths)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [21]:
# ALL CONTEXT VECTORS
all_context_vecs = attn_weigths @ inputs
print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


## Self Attention with trainabel weights

In [23]:
x_2 = inputs[1]
d_in = inputs.shape[1]
d_out = 2

In [24]:
torch.manual_seed(123)
w_query = torch.nn.Parameter(torch.rand(d_in,d_out), requires_grad = False)
w_key = torch.nn.Parameter(torch.rand(d_in,d_out), requires_grad = False)
w_value = torch.nn.Parameter(torch.rand(d_in,d_out), requires_grad = False)

In [25]:
print(w_query.shape)

torch.Size([3, 2])


In [26]:
w_query

Parameter containing:
tensor([[0.2961, 0.5166],
        [0.2517, 0.6886],
        [0.0740, 0.8665]])

In [27]:
query_2 = x_2 @ w_query
key_2 = x_2 @ w_key
value_2 = x_2 @ w_value
print(query_2)

tensor([0.4306, 1.4551])


In [28]:
keys = inputs@ w_key
values = inputs @ w_value

print(keys.shape)

torch.Size([6, 2])


In [29]:
# calculating atten score for query 2
keys_2 = keys[1]
attn_score_2 = query_2.dot(keys_2)
print(attn_score_2)

tensor(1.8524)


In [30]:
attn_scores_2 = query_2 @ keys.T
print(attn_scores_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


In [31]:
# scaled dot product 
d_k = keys.shape[-1]
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5 , dim = -1)
print(attn_weights_2)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


In [32]:
context_vec_2 = attn_weights_2 @ values
print(context_vec_2)

tensor([0.3061, 0.8210])


## Self Attention Class

In [33]:
import torch.nn as nn

class SelfAttention_v1(nn.Module):

    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key   = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x):
        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value

        attn_scores = queries @ keys.T
        attn_weights = torch.softmax( attn_scores/ keys.shape[-1]**0.5 , dim=-1)

        context_vector = attn_weights @ values

        return context_vector

torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


## Self Attention usign Pytorch Linear Layer

In [35]:
class SelfAttention_v2(nn.Module):

    def __init__(self, d_in, d_out, qkv_bias = False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias = qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias = qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias = qkv_bias)

    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.T
        attn_weights = torch.softmax( attn_scores/ keys.shape[-1]**0.5 , dim=-1)

        context_vector = attn_weights @ values

        return context_vector

torch.manual_seed(234)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

tensor([[0.5922, 0.2749],
        [0.5869, 0.2702],
        [0.5864, 0.2698],
        [0.5885, 0.2713],
        [0.5778, 0.2628],
        [0.5935, 0.2753]], grad_fn=<MmBackward0>)


In [38]:
state_dict_v2 = sa_v2.state_dict()
print(state_dict_v2.keys())

odict_keys(['W_query.weight', 'W_key.weight', 'W_value.weight'])


In [39]:
# Extract specific weights from sa_v2 model
W_query_weights = state_dict_v2['W_query.weight']
W_key_weights = state_dict_v2['W_key.weight']
W_value_weights = state_dict_v2['W_value.weight']

print(W_query_weights.shape)  # transposed form

torch.Size([2, 3])


In [41]:
state_dict_v1 = sa_v1.state_dict()
print(state_dict_v1.keys())

# Extract specific weights
W_query_weights_v1 = state_dict_v1['W_query']

print(W_query_weights_v1.shape)

odict_keys(['W_query', 'W_key', 'W_value'])
torch.Size([3, 2])


In [42]:
sa_v1.W_query.data = W_query_weights.T
sa_v1.W_key.data = W_key_weights.T
sa_v1.W_value.data = W_value_weights.T

In [44]:
print(sa_v1(inputs)) # now it prints same result for both SA_v1 and sa_v2

tensor([[0.5922, 0.2749],
        [0.5869, 0.2702],
        [0.5864, 0.2698],
        [0.5885, 0.2713],
        [0.5778, 0.2628],
        [0.5935, 0.2753]], grad_fn=<MmBackward0>)


## Causal Attention

In [54]:
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs) 
attn_scores = queries @ keys.T

attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[0.1657, 0.1713, 0.1710, 0.1641, 0.1598, 0.1682],
        [0.1702, 0.1642, 0.1642, 0.1676, 0.1667, 0.1672],
        [0.1710, 0.1634, 0.1634, 0.1679, 0.1670, 0.1673],
        [0.1666, 0.1665, 0.1665, 0.1668, 0.1671, 0.1666],
        [0.1856, 0.1511, 0.1513, 0.1728, 0.1712, 0.1680],
        [0.1587, 0.1737, 0.1736, 0.1638, 0.1645, 0.1658]],
       grad_fn=<SoftmaxBackward0>)


In [55]:
print(attn_scores)

tensor([[ 0.0987,  0.1465,  0.1434,  0.0851,  0.0478,  0.1199],
        [-0.0204, -0.0712, -0.0711, -0.0418, -0.0491, -0.0452],
        [-0.0287, -0.0930, -0.0928, -0.0546, -0.0623, -0.0600],
        [-0.0051, -0.0059, -0.0057, -0.0034, -0.0008, -0.0054],
        [-0.1686, -0.4594, -0.4568, -0.2691, -0.2824, -0.3093],
        [ 0.0737,  0.2016,  0.2005,  0.1181,  0.1242,  0.1356]],
       grad_fn=<MmBackward0>)


In [48]:
context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))
print(mask_simple)


tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


In [49]:
masked_simple = attn_weights * mask_simple
print(masked_simple)

tensor([[0.1657, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1702, 0.1642, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1710, 0.1634, 0.1634, 0.0000, 0.0000, 0.0000],
        [0.1666, 0.1665, 0.1665, 0.1668, 0.0000, 0.0000],
        [0.1856, 0.1511, 0.1513, 0.1728, 0.1712, 0.0000],
        [0.1587, 0.1737, 0.1736, 0.1638, 0.1645, 0.1658]],
       grad_fn=<MulBackward0>)


In [50]:
row_sums = masked_simple.sum(dim =-1, keepdim = True)
masked_simple_norm = masked_simple / row_sums
print(masked_simple_norm)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5090, 0.4910, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3435, 0.3282, 0.3283, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2498, 0.2499, 0.2503, 0.0000, 0.0000],
        [0.2230, 0.1816, 0.1819, 0.2077, 0.2058, 0.0000],
        [0.1587, 0.1737, 0.1736, 0.1638, 0.1645, 0.1658]],
       grad_fn=<DivBackward0>)


In [52]:
mask = torch.triu(torch.ones(context_length, context_length), diagonal = 1)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
print(masked)

tensor([[ 0.0987,    -inf,    -inf,    -inf,    -inf,    -inf],
        [-0.0204, -0.0712,    -inf,    -inf,    -inf,    -inf],
        [-0.0287, -0.0930, -0.0928,    -inf,    -inf,    -inf],
        [-0.0051, -0.0059, -0.0057, -0.0034,    -inf,    -inf],
        [-0.1686, -0.4594, -0.4568, -0.2691, -0.2824,    -inf],
        [ 0.0737,  0.2016,  0.2005,  0.1181,  0.1242,  0.1356]],
       grad_fn=<MaskedFillBackward0>)


In [56]:
attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim = -1)
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5090, 0.4910, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3435, 0.3282, 0.3283, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2498, 0.2499, 0.2503, 0.0000, 0.0000],
        [0.2230, 0.1816, 0.1819, 0.2077, 0.2058, 0.0000],
        [0.1587, 0.1737, 0.1736, 0.1638, 0.1645, 0.1658]],
       grad_fn=<SoftmaxBackward0>)


In [59]:
context_vec = attn_weights @ values
print(context_vec)

tensor([[0.1855, 0.8812],
        [0.2884, 0.9414],
        [0.3208, 0.9549],
        [0.3019, 0.8542],
        [0.2641, 0.7406],
        [0.2825, 0.7602]], grad_fn=<MmBackward0>)


In [61]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5)
example = torch.ones(6,6)
print(dropout(example))

tensor([[2., 2., 0., 2., 2., 0.],
        [0., 0., 0., 2., 0., 2.],
        [2., 2., 2., 2., 0., 2.],
        [0., 2., 2., 0., 0., 2.],
        [0., 2., 0., 2., 0., 2.],
        [0., 2., 2., 2., 2., 0.]])


In [63]:
torch.manual_seed(123)
print(dropout(attn_weights))

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.6870, 0.6564, 0.6566, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.4997, 0.4998, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3631, 0.0000, 0.4155, 0.0000, 0.0000],
        [0.0000, 0.3474, 0.3472, 0.3275, 0.3289, 0.0000]],
       grad_fn=<MulBackward0>)


## Compact Causal Attention Class

In [64]:
batch = torch.stack((inputs, inputs), dim = 0)
print(batch.shape)

torch.Size([2, 6, 3])


In [65]:
class CausalAttention(nn.Module):

    def __init__(self, d_in, d_out, context_length, dropout , qkv_bias = False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias = qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias = qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias = qkv_bias)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal = 1))

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1,2)
        attn_scores.masked_fill(self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)
        attn_weights = torch.softmax( attn_scores/ keys.shape[-1]**0.5 , dim=-1)

        attn_weights = self.dropout(attn_weights)
        
        context_vector = attn_weights @ values

        return context_vector


In [66]:
torch.manual_seed(123)
context_length = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_length, 0.0)
context_vecs = ca(batch)
print(context_vecs.shape)

torch.Size([2, 6, 2])


## Multi Head Attention 

In [72]:
class MultiHeadAttentionWrapper(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias = False):
        super().__init__()
        self.heads = nn.ModuleList([
            CausalAttention(d_in, d_out, context_length, dropout, qkv_bias) for i in range(num_heads)
        ])
    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim = -1)
        

In [75]:
torch.manual_seed(123)
context_length = batch.shape[1]
d_in, d_out = 3 , 2
mha = MultiHeadAttentionWrapper(d_in, d_out, context_length, 0.0, num_heads =2)
context_vecs = mha(batch)
print(context_vecs.shape)

torch.Size([2, 6, 4])


In [82]:
#Combining it all together

class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias =False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

        self.out_proj = nn.Linear(d_out, d_out)
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal = 1))

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        print('x',x.shape)
        print('key',self.W_key)

        keys = self.W_key(x)
        print('keys',keys.shape)
        queries = self.W_query(x)
        values = self.W_query(x)

        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        print('values',values.shape)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        keys = keys.transpose(1,2)
        values = values.transpose(1,2)
        print('values',values.shape)
        queries = queries.transpose(1,2)

        attn_scores = queries @ keys.transpose(2,3)
        print('attn_scores',attn_scores.shape)

        mask_bool = self.mask.bool()[:num_tokens, : num_tokens]
        attn_scores.masked_fill(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores/keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)
        print('attn_weights',attn_weights.shape)

        context_vec = (attn_weights @ values).transpose(1,2)
        print('context_vec',context_vec.shape)
        context_vec = context_vec.contiguous().view(b,num_tokens,self.d_out)
        print('context_vec',context_vec.shape)
        context_vec = self.out_proj(context_vec)
        print('context_vec',context_vec.shape)

        return context_vec

torch.manual_seed(123)
batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

x torch.Size([2, 6, 3])
key Linear(in_features=3, out_features=2, bias=False)
keys torch.Size([2, 6, 2])
values torch.Size([2, 6, 2, 1])
values torch.Size([2, 2, 6, 1])
attn_scores torch.Size([2, 2, 6, 6])
attn_weights torch.Size([2, 2, 6, 6])
context_vec torch.Size([2, 6, 2, 1])
context_vec torch.Size([2, 6, 2])
context_vec torch.Size([2, 6, 2])
tensor([[[0.2427, 0.5651],
         [0.2409, 0.5643],
         [0.2410, 0.5643],
         [0.2405, 0.5648],
         [0.2413, 0.5650],
         [0.2403, 0.5646]],

        [[0.2427, 0.5651],
         [0.2409, 0.5643],
         [0.2410, 0.5643],
         [0.2405, 0.5648],
         [0.2413, 0.5650],
         [0.2403, 0.5646]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


In [84]:
context_length = 1024
d_in, d_out = 768, 768
num_heads = 12
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads)
context_vecs = mha(batch)
print("context_vecs.shape:", context_vecs.shape)

x torch.Size([2, 6, 3])
key Linear(in_features=768, out_features=768, bias=False)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (12x3 and 768x768)